In [1]:
library(tidyverse); library(zoo); library(sqldf)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0.9000     ✔ purrr   0.3.4     
✔ tibble  3.0.0          ✔ dplyr   0.8.5     
✔ tidyr   1.0.2          ✔ stringr 1.4.0     
✔ readr   1.3.1          ✔ forcats 0.5.0     

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: gsubfn

Loading required package: proto

Warning message:
“no DISPLAY variable so Tk is not available”
Loading required package: RSQLite



[SageMathCell Version](https://olgabelitskaya.github.io/sql_cookbook.html) & [SageMathCell Test Example](https://olgabelitskaya.github.io/sql_test.html)
## 📑 Creating SQL Databases

In [2]:
if (!is.null(getOption("sqldf.connection"))) sqldf()
sqlite<-dbDriver("SQLite")
example<-dbConnect(sqlite,dbname="example.db")
for (df in c("projects","tasks","df1","df2",
             "tips","schooledu","nba")) {
    if (dbExistsTable(example,df)) {
        dbRemoveTable(example,df)}}
list.files(path="../input")

[1] "data-science-for-good"      "nba-all-star-game-20002016"

## 📑 Creating SQL Tables

In [3]:
st_ex<-function(str){
    st<-do.call(paste,c(as.list(str),sep=""))
    dbExecute(st,conn=example)}
cr_task<-function(str){
    st<-c("INSERT INTO tasks(name,priority,status_id,",
          "project_id,begin_date,end_date) VALUES",str)
    st<-do.call(paste,c(as.list(st),sep=""))
    dbExecute(st,conn=example)}

In [4]:
st1<-c("CREATE TABLE IF NOT EXISTS projects (",
       "id integer PRIMARY KEY, ",
       "name text NOT NULL, ",
       " begin_date text, ",
       " end_date text);")
st2<-c("CREATE TABLE IF NOT EXISTS tasks ( ",
       "id integer PRIMARY KEY, ",
       "name text NOT NULL, ",
       "priority integer, ",
       "status_id integer NOT NULL, ",
       "project_id integer NOT NULL, ",
       "begin_date text NOT NULL, ",
       "end_date text NOT NULL, ",
       "FOREIGN KEY (project_id) REFERENCES projects (id));")
st3<-c("INSERT INTO projects(name,begin_date,end_date) ",
       "VALUES('SQL CookBook with SageMathCell',",
       "'2020-02-04','2020-02-18');")
st4<-c("INSERT INTO projects(name,begin_date,end_date) ",
       "VALUES('SQL Tests','2020-01-31','2020-02-14');")
st5<-list("('Page 1',1,1,1,'2020-02-04','2020-02-12');",
          "('Page 2',1,1,1,'2020-02-10','2020-02-18');",
          "('Test 1',1,1,2,'2020-01-31','2020-02-01');",
          "('Test 2',1,1,2,'2020-02-02','2020-02-03');",
          "('Test 3',1,1,2,'2020-02-04','2020-02-05');",
          "('Test 4',1,1,2,'2020-02-06','2020-02-07');",
          "('Test 5',1,1,2,'2020-02-08','2020-02-09');",
          "('Test 6',1,1,2,'2020-02-09','2020-02-10');",
          "('Test 7',1,1,2,'2020-02-10','2020-02-11');",
          "('Dublicate',1,1,2,'2020-02-11','2020-02-14')")
for (st in list(st1,st2,st3,st4)) {st_ex(st)}
for (st in st5) {cr_task(st)}

## 📑 Creating DataFrames

In [5]:
df1<-data.frame(key1=c('A','B','C','D','F','F'),
                value1=sample(1:10,6))
df2<-data.frame(key2=c('B','D','D','E','F'),
                value2=sample(1:10,5))
df1; df2

key1,value1
<fct>,<int>
A,2
B,4
C,5
D,6
F,8
F,10


key2,value2
<fct>,<int>
B,4
D,6
D,3
E,8
F,5


In [6]:
path1<-paste0('https://raw.github.com/pydata/pandas/',
              'master/pandas/tests/data/')
tips<-read.csv(paste0(path1,'tips.csv'))

In [7]:
path2<-'../input/data-science-for-good/'
school_explorer<-read.csv(paste0(path2,'2016 School Explorer.csv'))

In [8]:
school_explorer<-school_explorer[,4:161]
school_explorer[school_explorer=="N/A"]<-NA
school_explorer$School.Name<-
as.character(school_explorer$School.Name)
school_explorer$School.Name[c(428,1024,713,909)]<- 
c('P.S. 212 D12','P.S. 212 D30','P.S. 253 D21','P.S. 253 D27')
school_explorer$School.Income.Estimate<-
as.character(school_explorer$School.Income.Estimate)
school_explorer$School.Income.Estimate<-
sub("\\$","",school_explorer$School.Income.Estimate)
school_explorer$School.Income.Estimate<-
sub(",","",school_explorer$School.Income.Estimate)
school_explorer$School.Income.Estimate<-
as.numeric(school_explorer$School.Income.Estimate)
school_explorer$School.Income.Estimate<-
na.approx(school_explorer$School.Income.Estimate)
percent_list<-c('Percent.ELL',
                'Percent.Asian','Percent.Black','Percent.Hispanic',
                'Percent.Black...Hispanic','Percent.White',
                'Student.Attendance.Rate',
                'Percent.of.Students.Chronically.Absent',
                'Rigorous.Instruction..','Collaborative.Teachers..',
                'Supportive.Environment..','Effective.School.Leadership..',
                'Strong.Family.Community.Ties..','Trust..')
target_list<-c("Average.ELA.Proficiency","Average.Math.Proficiency")
economic_list<-c("Economic.Need.Index","School.Income.Estimate")
rating_list<-c('Rigorous.Instruction.Rating','Collaborative.Teachers.Rating',
               'Supportive.Environment.Rating','Effective.School.Leadership.Rating',
               'Strong.Family.Community.Ties.Rating','Trust.Rating',
               'Student.Achievement.Rating')
nastr2num<-function(x){ x<-as.numeric(sub("%","",x)) }
for (el in c(percent_list,target_list,"Economic.Need.Index")) {
    school_explorer[el]<-sapply(school_explorer[el],nastr2num)
    school_explorer[el]<-na.approx(school_explorer[el]) }
for (el in rating_list) {
    vtable<-data.frame(table(school_explorer[el]))
    mvalue<-as.character(vtable$Var1[vtable$Freq==max(vtable$Freq)])
    school_explorer[el][is.na(school_explorer[el])]<-mvalue
    school_explorer[,el]<-factor(school_explorer[,el]) }
sum(is.na(school_explorer))

[1] 0

In [9]:
names(school_explorer)<-gsub("\\.","",names(school_explorer))
head(school_explorer)[1:10]

,SchoolName,SEDCode,LocationCode,District,Latitude,Longitude,AddressFull,City,Zip,Grades
,<chr>,<dbl>,<fct>,<int>,<dbl>,<dbl>,<fct>,<fct>,<int>,<fct>
1,P.S. 015 ROBERTO CLEMENTE,3.101e+11,01M015,1,40.72183,-73.97877,"333 E 4TH ST NEW YORK, NY 10009",NEW YORK,10009,"PK,0K,01,02,03,04,05"
2,P.S. 019 ASHER LEVY,3.101e+11,01M019,1,40.72989,-73.98423,"185 1ST AVE NEW YORK, NY 10003",NEW YORK,10003,"PK,0K,01,02,03,04,05"
3,P.S. 020 ANNA SILVER,3.101e+11,01M020,1,40.72127,-73.98632,"166 ESSEX ST NEW YORK, NY 10002",NEW YORK,10002,"PK,0K,01,02,03,04,05"
4,P.S. 034 FRANKLIN D. ROOSEVELT,3.101e+11,01M034,1,40.72615,-73.97504,"730 E 12TH ST NEW YORK, NY 10009",NEW YORK,10009,"PK,0K,01,02,03,04,05,06,07,08"
5,THE STAR ACADEMY - P.S.63,3.101e+11,01M063,1,40.72440,-73.98636,"121 E 3RD ST NEW YORK, NY 10009",NEW YORK,10009,"PK,0K,01,02,03,04,05"
6,P.S. 064 ROBERT SIMON,3.101e+11,01M064,1,40.72375,-73.98160,"600 E 6TH ST NEW YORK, NY 10009",NEW YORK,10009,"PK,0K,01,02,03,04,05"


In [10]:
path3<-'../input/nba-all-star-game-20002016/'
fn3<-'NBA All Stars 2000-2016 - Sheet1.csv'
nba<-read.csv(paste0(path3,fn3))

## 📑 SQL Basic Queries

In [11]:
q<-paste0("SELECT * FROM projects,tasks ",
          "WHERE projects.id=tasks.project_id;")
sqldf(q,conn=example)

id,name,begin_date,end_date,id,name,priority,status_id,project_id,begin_date,end_date
<int>,<chr>,<chr>,<chr>,<int>,<chr>,<int>,<int>,<int>,<chr>,<chr>
1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,1,Page 1,1,1,1,2020-02-04,2020-02-12
1,SQL CookBook with SageMathCell,2020-02-04,2020-02-18,2,Page 2,1,1,1,2020-02-10,2020-02-18
2,SQL Tests,2020-01-31,2020-02-14,3,Test 1,1,1,2,2020-01-31,2020-02-01
2,SQL Tests,2020-01-31,2020-02-14,4,Test 2,1,1,2,2020-02-02,2020-02-03
2,SQL Tests,2020-01-31,2020-02-14,5,Test 3,1,1,2,2020-02-04,2020-02-05
2,SQL Tests,2020-01-31,2020-02-14,6,Test 4,1,1,2,2020-02-06,2020-02-07
2,SQL Tests,2020-01-31,2020-02-14,7,Test 5,1,1,2,2020-02-08,2020-02-09
2,SQL Tests,2020-01-31,2020-02-14,8,Test 6,1,1,2,2020-02-09,2020-02-10
2,SQL Tests,2020-01-31,2020-02-14,9,Test 7,1,1,2,2020-02-10,2020-02-11


In [12]:
q<-"SELECT * from df1,df2;"
sqldf(q)

key1,value1,key2,value2
<fct>,<int>,<fct>,<int>
A,2,B,4
A,2,D,6
A,2,D,3
A,2,E,8
A,2,F,5
B,4,B,4
B,4,D,6
B,4,D,3
B,4,E,8


In [13]:
q<-c("SELECT * from df1,df2 ",
     "WHERE df1.key1=df2.key2 AND df1.value1>2 ",
     "ORDER BY key1 DESC;")
q<-do.call(paste,c(as.list(q),sep=""))
sqldf(q)

key1,value1,key2,value2
<fct>,<int>,<fct>,<int>
F,8,F,5
F,10,F,5
D,6,D,3
D,6,D,6
B,4,B,4


In [14]:
q<-paste0("SELECT total_bill,tip,smoker,time ",
          "FROM tips LIMIT 7;")
sqldf(q)

total_bill,tip,smoker,time
<dbl>,<dbl>,<fct>,<fct>
16.99,1.01,No,Dinner
10.34,1.66,No,Dinner
21.01,3.50,No,Dinner
23.68,3.31,No,Dinner
24.59,3.61,No,Dinner
25.29,4.71,No,Dinner
8.77,2.00,No,Dinner


In [15]:
q<-paste0("SELECT * FROM tips ",
          "WHERE time='Dinner' LIMIT 3;")
sqldf(q)

total_bill,tip,sex,smoker,day,time,size
<dbl>,<dbl>,<fct>,<fct>,<fct>,<fct>,<int>
16.99,1.01,Female,No,Sun,Dinner,2
10.34,1.66,Male,No,Sun,Dinner,3
21.01,3.50,Male,No,Sun,Dinner,3


In [16]:
q<-paste0("SELECT COUNT(LocationCode),TrustRating ",
          "FROM school_explorer GROUP BY TrustRating;")
sqldf(q)

COUNT(LocationCode),TrustRating
<int>,<fct>
208,Approaching Target
364,Exceeding Target
669,Meeting Target
31,Not Meeting Target


In [17]:
q<-"PRAGMA table_info('nba');"
sqldf(q)

cid,name,type,notnull,dflt_value,pk
<int>,<chr>,<chr>,<int>,<lgl>,<int>
0,Year,INTEGER,0,NA,0
1,Player,TEXT,0,NA,0
2,Pos,TEXT,0,NA,0
3,HT,TEXT,0,NA,0
4,WT,INTEGER,0,NA,0
5,Team,TEXT,0,NA,0
6,Selection.Type,TEXT,0,NA,0
7,NBA.Draft.Status,TEXT,0,NA,0
8,Nationality,TEXT,0,NA,0


In [18]:
q<-"SELECT DISTINCT Team FROM nba WHERE Year=2014;"
sqldf(q)

Team
<fct>
Cleveland Cavaliers
Washington Wizards
Toronto Raptors
Chicago Bulls
New York Knicks
Atlanta Hawks
Miami Heat
Golden State Warriors
Los Angeles Lakers


## 📑 Closing the Connection

In [19]:
sqldf("SELECT * from sqlite_master;",
       conn=example)

type,name,tbl_name,rootpage,sql
<chr>,<chr>,<chr>,<int>,<chr>
table,projects,projects,2,"CREATE TABLE projects (id integer PRIMARY KEY, name text NOT NULL, begin_date text, end_date text)"
table,tasks,tasks,3,"CREATE TABLE tasks ( id integer PRIMARY KEY, name text NOT NULL, priority integer, status_id integer NOT NULL, project_id integer NOT NULL, begin_date text NOT NULL, end_date text NOT NULL, FOREIGN KEY (project_id) REFERENCES projects (id))"


In [20]:
dbDisconnect(example)
dbUnloadDriver(sqlite)
file.remove("example.db")

[1] TRUE